## MongoDB: Operaciones CRUD
 
Basicamente revisamos que las operaciones CRUD se puedan realizar sobre la colección en **MongoDB**. Realizamos una simulación de flujo de creación, lectura, edición y eliminación.

### Instalaciones

Descargamos las librerías necesarias para ejecutar las operaciones.

In [47]:
%pip install requests
%pip install pandas
%pip install pymongo
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importaciones

Realizamos las importaciones necesarias para ejecutar las operaciones

In [48]:
from pymongo import MongoClient
from dotenv import load_dotenv
import json
import pprint
import os
import sys
import subprocess
from bson.decimal128 import Decimal128
from statistics import mean

### Conexión a Cluster MongoDB

Nos conectamos al cluster creado en **MongoDB**. Debe tener un **.env** creado con la variable **MONGO_URI**.

In [49]:
load_dotenv()
uri = os.getenv("MONGO_URI")
client = MongoClient(uri)

### Conexión a Base de Datos y Colecciones

Nos conectamos a la colección **ProyectoBD-crud**, y a las siguientes colecciones:

- **dias**
- **metricas**
- **registros**
- **kpis**

Básicamente nos estamos conectando a una copia de la colección original **ProyectoBD** para poder realizar las pruebas de operaciones.

In [50]:
db = client["ProyectoBD-crud"]
dias = db["dias"]
metricas = db["metricas"]
registros = db["registros"]
kpis = db["kpis"]

### Limpieza

Básicamente vaciamos las colecciones para repoblarlas, realizando la **creacion**, y luego probar las operaciones de **lectura**, **edición** y **eliminación**.

##### 1. Vacío de las colecciones

Borramos todos los registros en cada colección.

In [51]:
db.dias.delete_many({});
db.metricas.delete_many({});
db.registros.delete_many({});

##### 2. Comprobación de limpieza

Verificamos que no hayan registros en ninguna colección.

In [52]:
total_dias = dias.count_documents({})
print(f"Total de registros en dias: {total_dias}")

total_metricas = metricas.count_documents({})
print(f"Total de registros en métricas: {total_metricas}")

total_registros = registros.count_documents({})
print(f"Total de registros en registros: {total_registros}")

Total de registros en dias: 0
Total de registros en métricas: 0
Total de registros en registros: 0


### Creación: Procesamiento de la data e inserción

Como datos para prueba insertamos los movimientos de **GitHub** de la semana del **2024-12-30 al 2025-01-05**, de lunes a domingo para los siguientes rangos horarios:

- **00:00 - 01:00**
- **01:00 - 02:00**

El tipo de datos es identico para los demás rangos horarios, por lo tanto solo probamos con dos. Ello debido al tiempo que tomaría analizar más rangos. 

La data de los movimientos de **GitHub** proviene de *https://data.gharchive.org/*, y por ejemplo para los movimiento del dia **2024-12-30** en el rango horario **00:00-01:00** descargamos los datos de *https://data.gharchive.org/2024-12-30-0.json.gz*. Para cada día, en este caso incluyendo solo los rangos horarios anteriores, extraemos lo siguiente:

- **gh_dias.json**: Extrae el resumen del día; se inserta en la colección **dias**
- **gh_muestras.json**: Extrae y procesa una muestra del dia procesado. Se ha designado 209 registros por rango horario. Por tanto en este caso, 418 registros por dia. Se insertan en la colección **registros**
- **metric.json**: Extrae métricas representativas del día; se inserta en la colección **metricas**

Puede revisar los archivos resultantes de la ejecución del siguiente código del último día procesado, es decir **2025-01-05**.

In [53]:
python_exec = sys.executable
script = "process_data.py"

args = ["2024-12-30", "2024-12-31", "2025-01-01", "2025-01-02", "2025-01-03", "2025-01-04", "2025-01-05"]

for arg in args:
    subprocess.run([python_exec, script, arg])

    with open("gh_dias.json") as f:
        dia = json.load(f)
        dias.insert_one(dia)
    
    with open("metrics.json") as f:
        metrics = json.load(f)
        metricas.insert_one(metrics)

    with open("gh_muestras.json") as f:
        registers = json.load(f)
        registros.insert_many(registers)

print(f"\n{args[0]} al {args[len(args)-1]} procesado")



📅 Procesando 2024-12-30...


📅 Procesando 2024-12-31...


📅 Procesando 2025-01-01...


📅 Procesando 2025-01-02...


📅 Procesando 2025-01-03...


📅 Procesando 2025-01-04...


📅 Procesando 2025-01-05...


2024-12-30 al 2025-01-05 procesado


### Lectura

Validamos que evidentemente se hayan creado los registros anteriores, y luego hacemos una consulta básica en cada colección para simular lecturas.

##### 1. Comprobación de creación

In [54]:
total_dias = dias.count_documents({})
print(f"Total de registros en dias: {total_dias}")

total_metricas = metricas.count_documents({})
print(f"Total de registros en métricas: {total_metricas}")

total_registros = registros.count_documents({})
print(f"Total de registros en registros: {total_registros}")

Total de registros en dias: 7
Total de registros en métricas: 7
Total de registros en registros: 2926


##### 2. Ejemplo de registros

Realizamos una lectura básica por colección, y luego una específica.

- **Dias**: Hacemos una busqueda sin filtro, y luego por días en específico.

In [55]:
print("Dia: Ejemplo")
dia = dias.find_one({})
pprint.pprint(dia)

print(f"\nDía: Diferentes a {dia['dia']}")
cantidad = dias.count_documents({"dia": {"$ne": dia["dia"]}})

print(f"Cantidad: {cantidad} - Se imprime uno solo")
consulta = dias.find_one({"dia": {"$ne": dia["dia"]}})

pprint.pprint(consulta)


Dia: Ejemplo
{'_id': ObjectId('692332efa742ecd28945d538'),
 'cantidad_archivos': 2,
 'cantidad_regs': 355441,
 'cantidad_regs_sample': 418,
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'tipos_eventos': ['CommitCommentEvent',
                   'CreateEvent',
                   'DeleteEvent',
                   'ForkEvent',
                   'GollumEvent',
                   'IssueCommentEvent',
                   'IssuesEvent',
                   'MemberEvent',
                   'PublicEvent',
                   'PullRequestEvent',
                   'PullRequestReviewCommentEvent',
                   'PullRequestReviewEvent',
                   'PushEvent',
                   'ReleaseEvent',
                   'WatchEvent']}

Día: Diferentes a Lunes
Cantidad: 6 - Se imprime uno solo
{'_id': ObjectId('6923332aa742ecd28945d6dc'),
 'cantidad_archivos': 2,
 'cantidad_regs': 359745,
 'cantidad_regs_sample': 418,
 'dia': 'Martes',
 'fecha': '2024-12-31',
 'tipos_eventos': ['CommitCommentEven

- **Metricas**: Hacemos una busqueda sin filtro, y luego por actor más recurrente en específico

In [56]:
print("Metricas: Ejemplo")
metrica = metricas.find_one({})
pprint.pprint(metrica)

print(f"\nMetricas: Actor más recurrente igual a '{metrica["actor_mas_recurrente"]}'")
cantidad = metricas.count_documents({"actor_mas_recurrente": {"$eq": metrica["actor_mas_recurrente"]}})

print(f"Cantidad: {cantidad} - Se imprime uno solo")
consulta = metricas.find_one({"actor_mas_recurrente": {"$eq": metrica["actor_mas_recurrente"]}})

pprint.pprint(consulta)


Metricas: Ejemplo
{'_id': ObjectId('692332efa742ecd28945d539'),
 'accion_mas_recurrente': 'PushEvent',
 'actor_mas_recurrente': 'github-actions[bot]',
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'horas_inactivas': [2,
                     3,
                     4,
                     5,
                     6,
                     7,
                     8,
                     9,
                     10,
                     11,
                     12,
                     13,
                     14,
                     15,
                     16,
                     17,
                     18,
                     19,
                     20,
                     21,
                     22,
                     23]}

Metricas: Actor más recurrente igual a 'github-actions[bot]'
Cantidad: 7 - Se imprime uno solo
{'_id': ObjectId('692332efa742ecd28945d539'),
 'accion_mas_recurrente': 'PushEvent',
 'actor_mas_recurrente': 'github-actions[bot]',
 'dia': 'Lunes',
 'fecha': '2024-12-

- **Registros**: Hacemos una busqueda sin filtro, y luego por tamaño de push en específico

In [57]:
print("Registro: Ejemplo")
registro = registros.find_one({})
pprint.pprint(registro)

print(f"\nRegistros: Tamaño de push mayor a {registro["tamano_push"]}")
cantidad = registros.count_documents({"tamano_push": {"$gte": registro["tamano_push"]}})

print(f"Cantidad: {cantidad} - Se imprime uno solo")
consulta = registros.find_one({"tamano_push": {"$gte": registro["tamano_push"]}})

pprint.pprint(consulta)


Registro: Ejemplo
{'_id': ObjectId('692332efa742ecd28945d53a'),
 'action': None,
 'actor_id': 41898282,
 'actor_login': 'github-actions[bot]',
 'branch': 'rss-md',
 'created_at': '2024-12-30 00:25:44+00:00',
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'hora': '00:00-01:00',
 'n_commits': 1,
 'repo_id': 281825373,
 'repo_name': 'simeononsecurity/simeononsecurity.ch',
 'tamano_push': 1.0,
 'type': 'PushEvent'}

Registros: Tamaño de push mayor a 1.0
Cantidad: 2093 - Se imprime uno solo
{'_id': ObjectId('692332efa742ecd28945d53a'),
 'action': None,
 'actor_id': 41898282,
 'actor_login': 'github-actions[bot]',
 'branch': 'rss-md',
 'created_at': '2024-12-30 00:25:44+00:00',
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'hora': '00:00-01:00',
 'n_commits': 1,
 'repo_id': 281825373,
 'repo_name': 'simeononsecurity/simeononsecurity.ch',
 'tamano_push': 1.0,
 'type': 'PushEvent'}


### Actualización

Realizamos una actualización en cada colección, simulando por ejemplo una limpieza. En cada colección realizamos una actualización diferente.

- **Dias**: Actualizamos los dias indicando el porcentaje del total que representa la muestra

In [58]:
consulta = dias.find({})

for dia in consulta:
    porcentaje = (dia["cantidad_regs_sample"] / dia["cantidad_regs"]) * 100

    dias.update_one(
        {"_id": {"$eq": dia["_id"]}},
        {"$set": {"porcentaje_sample": porcentaje}}
    )

muestra = dias.find_one({})
print("Dia: Luego de actualización")
pprint.pprint(muestra)


Dia: Luego de actualización
{'_id': ObjectId('692332efa742ecd28945d538'),
 'cantidad_archivos': 2,
 'cantidad_regs': 355441,
 'cantidad_regs_sample': 418,
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'porcentaje_sample': 0.1176003893754519,
 'tipos_eventos': ['CommitCommentEvent',
                   'CreateEvent',
                   'DeleteEvent',
                   'ForkEvent',
                   'GollumEvent',
                   'IssueCommentEvent',
                   'IssuesEvent',
                   'MemberEvent',
                   'PublicEvent',
                   'PullRequestEvent',
                   'PullRequestReviewCommentEvent',
                   'PullRequestReviewEvent',
                   'PushEvent',
                   'ReleaseEvent',
                   'WatchEvent']}


- **Metricas**: Actualizamos las métricas reemplazando **github-actions[bot]** por **bot**

In [59]:
result = metricas.update_many(
    {"actor_mas_recurrente": "github-actions[bot]"},
    {"$set": {"actor_mas_recurrente": "bot"}}
)

muestra = metricas.find_one({})
print("Metrica: Luego de actualización")
pprint.pprint(muestra)

Metrica: Luego de actualización
{'_id': ObjectId('692332efa742ecd28945d539'),
 'accion_mas_recurrente': 'PushEvent',
 'actor_mas_recurrente': 'bot',
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'horas_inactivas': [2,
                     3,
                     4,
                     5,
                     6,
                     7,
                     8,
                     9,
                     10,
                     11,
                     12,
                     13,
                     14,
                     15,
                     16,
                     17,
                     18,
                     19,
                     20,
                     21,
                     22,
                     23]}


- **Registros**: Repetimos la misma estrategia que en **métricas**, reemplazando **github-actions[bot]** por **bot**

In [60]:
result = registros.update_many(
    {"actor_login": "github-actions[bot]"},
    {"$set": {"actor_login": "bot"}}
)

muestra = registros.find_one({})
print("Registro: Luego de actualización")
pprint.pprint(muestra)


Registro: Luego de actualización
{'_id': ObjectId('692332efa742ecd28945d53a'),
 'action': None,
 'actor_id': 41898282,
 'actor_login': 'bot',
 'branch': 'rss-md',
 'created_at': '2024-12-30 00:25:44+00:00',
 'dia': 'Lunes',
 'fecha': '2024-12-30',
 'hora': '00:00-01:00',
 'n_commits': 1,
 'repo_id': 281825373,
 'repo_name': 'simeononsecurity/simeononsecurity.ch',
 'tamano_push': 1.0,
 'type': 'PushEvent'}


### Eliminación

Realizamos una eliminación en cada colección, simulando una limpieza. Para todas las colecciones, borramos los documentos que apuntan al día **Lunes**.

- **Dias**

In [61]:
result = dias.delete_many({"dia": "Lunes"})
print(f"Dias eliminados: {result.deleted_count}")

dias_eliminados = dias.count_documents({"dia": {"$eq": "Lunes"}})
print(f"Lunes luego de eliminación {dias_eliminados}")

Dias eliminados: 1
Lunes luego de eliminación 0


- **Metricas**

In [62]:
result = metricas.delete_many({"dia": "Lunes"})
print(f"Metricas eliminadas: {result.deleted_count}")

metricas_eliminadas = metricas.count_documents({"dia": {"$eq": "Lunes"}})
print(f"Lunes luego de eliminación {metricas_eliminadas}")

Metricas eliminadas: 1
Lunes luego de eliminación 0


- **Registros**: También eliminamos los que tienen tamaño de push igual a **NaN**

In [63]:
result = registros.delete_many({"dia": "Lunes"})
print(f"Registros eliminados: {result.deleted_count}")

registros_eliminados = registros.count_documents({"dia": {"$eq": "Lunes"}})
print(f"Lunes luego de eliminación {registros_eliminados}")

result2 = registros.delete_many({"tamano_push": Decimal128("NaN")})
print("Registros eliminados con NaN en tamaño push:", result2.deleted_count)


Registros eliminados: 418
Lunes luego de eliminación 0
Registros eliminados con NaN en tamaño push: 0


### Cálculo: KPIs

Finalmente adicionamos una simulación del cálculo de los **KPI's** como se hace en la colección actual. Se recalca que estos se guardan en la colección **kpis** bajo un solo documento con el siguiente esquema:

```json
{
  "prom_actividad_real_por_dia": number,
  "prom_commits_muestra_por_dia": number,
  "prom_commits_muestra_por_hora": number,
  "prom_tamano_push_muestra_por_hora": number,
  "prom_tamano_push_muestra_por_dia": number,
  "actor_mas_recurrente_muestra": string,
  "evento_mas_recurrente_muestra": string,
}
```


- Limpieza de la colección: **kpis**

In [64]:
db.kpis.delete_one({})

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000000c0'), 'opTime': {'ts': Timestamp(1763914813, 1), 't': 192}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1763914813, 1), 'signature': {'hash': b'\x1e\xcf\xd2p\xf1\x1f\r\x0f\xf3M\x9e\xc4\x0c\xeb\xb7OX|@z', 'keyId': 7537752253990436871}}, 'operationTime': Timestamp(1763914813, 1)}, acknowledged=True)

- Promedio de actividad por dia

In [65]:
actividad_dia = [{"$group": { 
    "_id": None, 
    "prom_actividad_por_dia": {"$avg": "$cantidad_regs"}}}]

actividad_dia = list(dias.aggregate(actividad_dia))[0]["prom_actividad_por_dia"]

- Promedio de commits por hora

In [66]:
commits_hora = [
    {"$group": {
        "_id": {"fecha": "$fecha", "hora": "$hora"},
        "commits_por_hora_en_dia": {"$sum": "$n_commits"}
    }},

    {"$group": {
        "_id": "$_id.fecha",
        "promedio_commits_por_hora_del_dia": {"$avg": "$commits_por_hora_en_dia"}
    }},
    {"$group": {
        "_id": None,
        "promedio_commits_por_hora": {"$avg": "$promedio_commits_por_hora_del_dia"}
    }}
]

commits_hora = list(registros.aggregate(commits_hora))[0]["promedio_commits_por_hora"]

- Promedio de tamaño push por hora

In [67]:
tamano_push_hora = [
    {"$group": {
        "_id": {"fecha": "$fecha", "hora": "$hora"},
        "push_por_hora_en_dia": {"$sum": "$tamano_push"}
    }},
    {"$group": {
        "_id": "$_id.fecha",
        "promedio_push_por_hora_del_dia": {"$avg": "$push_por_hora_en_dia"}
    }},
    {"$group": {
        "_id": None,
        "prom_tamano_push_por_hora": {"$avg": "$promedio_push_por_hora_del_dia"}
    }}
]

tamano_push_hora = list(registros.aggregate(tamano_push_hora))[0]["prom_tamano_push_por_hora"]


- Actor mas recurrente

In [68]:
actor = [
    {"$group": {"_id": "$actor_login", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 1}
]

actor_mas_rec = list(registros.aggregate(actor))[0]["_id"]

- Evento mas recurrente

In [69]:
evento = [
    {"$group": {"_id": "$type", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 1}
]

evento_mas_rec = list(registros.aggregate(evento))[0]["_id"]

- Promedio tamaño de push por dia

In [70]:
tamano_push_dia = [
    {"$group": {
        "_id": "$fecha",
        "sum_push": {"$sum": "$tamano_push"}
    }},
    {"$group": {
        "_id": None,
        "prom_tamano_push_dia": {"$avg": "$sum_push"}
    }}
]

tamano_push_dia = list(registros.aggregate(tamano_push_dia))[0]["prom_tamano_push_dia"]

- Promedio de commits por dia

In [71]:
commits_dia = [
    {"$group": {
        "_id": "$fecha",
        "sum_commits": {"$sum": "$n_commits"}
    }},
    {"$group": {
        "_id": None,
        "prom_commits_por_dia": {"$avg": "$sum_commits"}
    }}
]

commits_dia = list(registros.aggregate(commits_dia))[0]["prom_commits_por_dia"]

- Inserción en la colección

In [72]:
kpis_insert = {"prom_actividad_real_por_dia": actividad_dia, 
               "prom_commits_muestra_por_dia": commits_dia,
               "prom_commits_muestra_por_hora": commits_hora,
               "prom_tamano_push_muestra_por_hora": tamano_push_hora,
               "prom_tamano_push_muestra_por_dia": tamano_push_dia,
               "actor_mas_recurrente_muestra": actor_mas_rec,
               "evento_mas_recurrente_muestra": evento_mas_rec}

kpis.insert_one(kpis_insert)

InsertOneResult(ObjectId('6923343ea742ecd28945e0b4'), acknowledged=True)

### Dashboard

A partir de todo lo simulado, plasmamos los resultados finales en un dashboard. Se puede acceder mediante el siguiente enlace:

- **https://charts.mongodb.com/charts-project-0-uacbwnm/public/dashboards/62231e4c-ccdd-4092-9be2-9de3222260de**

Cabe recalcar que este dashboard solo refleja lo ejecutado en esta simulación. Dejamos una imagen de la visualización.

![Imagen](imagenes/1.png)

### Desconexión

Nos desconectamos del cluster en MongoDB.

In [73]:
client.close()

### Exportación a HTML

Exportamos este registro a HTML para una mejor visualización.

In [75]:
!jupyter nbconvert --to html "mongo_crud.ipynb"

[NbConvertApp] Converting notebook mongo_crud.ipynb to html
[NbConvertApp] Writing 361955 bytes to mongo_crud.html
